In [1]:
# Esta celda configura el tema visual de la aplicación Streamlit.
# Crea la carpeta .streamlit si no existe y escribe un archivo config.toml con los colores y fuentes personalizados.
# Esto afecta la apariencia global de la app, haciéndola más acorde a la identidad visual deseada.

import os

os.makedirs(".streamlit", exist_ok=True)

with open(".streamlit/config.toml", "w") as f:
    f.write("""
[theme]
base="light"
primaryColor="#FC3055"
backgroundColor="#ffffff"
secondaryBackgroundColor="#f0f0f5"
textColor="#000000"
font="sans serif"
""")

In [2]:
# Esta celda define una magic personalizada para Jupyter/VSCode.
# Permite guardar el contenido de una celda en un archivo .py con codificación UTF-8 usando %%writefile.
# Es útil para modularizar el código y reutilizar funciones en otros scripts o notebooks.

from IPython.core.magic import register_cell_magic

@register_cell_magic
def writefile(line, cell):
    with open(line, 'w', encoding='utf-8') as f:
        f.write(cell)

In [3]:
%%writefile utils.py

# Este archivo contiene funciones utilitarias generales para la app Streamlit.
# Su objetivo es centralizar lógica común para el análisis y visualización de datos.

# Función: df_completitud
# - Calcula el porcentaje de órdenes que tienen costos de combustible, peajes y mantenimiento por periodo.
# - Devuelve un DataFrame agrupado por periodo con estos indicadores.
# - Útil para evaluar la calidad y completitud de los datos.

# Función: plot_completitud_y_mediana
# - Genera un gráfico de líneas con Plotly mostrando la completitud de los datos por componente y periodo.
# - Permite añadir líneas de promedio para comparar visualmente la completitud a lo largo del tiempo.
# - Facilita la identificación de periodos con baja calidad de datos.

# Función: show_info_columns
# - Calcula y muestra indicadores generales y estadísticos de las órdenes seleccionadas.
# - Incluye totales, promedios, medianas, cuartiles y máximos/mínimos de costos y kilómetros.
# - Presenta la información en un formato visual atractivo usando HTML y CSS embebido en Streamlit.
# - Ayuda a obtener una visión rápida y clara del estado de los datos filtrados.

import streamlit as st
import pandas as pd
import numpy as np

def df_completitud(df):
    df['Orden con Costo de Combustible'] = df['Costo Combustible'] > 0
    df['Orden con Costo de Peajes'] = df['Costo Peajes'] > 0
    df['Orden con Costo de Mantenimiento'] = df['Costo Mantenimiento'] > 0

    completitud_groupby = df.groupby(['Periodo']).agg({
        'No. Viajes':'sum',
        'Orden con Costo de Combustible': 'sum',
        'Orden con Costo de Peajes': 'sum',
        'Orden con Costo de Mantenimiento': 'sum'})

    completitud_groupby['% Órdenes con Costo Combustible'] = (completitud_groupby['Orden con Costo de Combustible'] / completitud_groupby['No. Viajes']) * 100
    completitud_groupby['% Órdenes con Costo Peajes'] = (completitud_groupby['Orden con Costo de Peajes'] / completitud_groupby['No. Viajes']) * 100
    completitud_groupby['% Órdenes con Costo Mantenimiento'] = (completitud_groupby['Orden con Costo de Mantenimiento'] / completitud_groupby['No. Viajes']) * 100

    return completitud_groupby

def plot_completitud_y_mediana(
    completitud_groupby, 
    columnas_estadistica,  # Lista de columnas para líneas de mediana
    dash_estadistica='dot',
    width = 1000,
    height = 700
    ):

    import plotly.graph_objects as go
    import numpy as np

    periodos = completitud_groupby.index.astype(str)
    componentes = [
        ('% Órdenes con Costo Combustible', 'Orden con Costo de Combustible', '#233ED9'),   # Azul fuerte
        ('% Órdenes con Costo Peajes', 'Orden con Costo de Peajes', '#F2CD5E'),             # Amarillo
        ('% Órdenes con Costo Mantenimiento', 'Orden con Costo de Mantenimiento', '#5086F2') # Azul claro
    ]

    fig = go.Figure()

    for i, (porcentaje, conteo_col, color) in enumerate(componentes):
        y = completitud_groupby[porcentaje]
        n = completitud_groupby[conteo_col]
        total = completitud_groupby['No. Viajes']
        # Asigna manualmente la posición para cada componente
        if i == 0:
            text_positions = ["top center"] * len(y)
        elif i == 1:
            text_positions = ["top center"] * len(y)
        else:
            text_positions = ["bottom center"] * len(y)
        fig.add_trace(go.Scatter(
            x=periodos,
            y=y,
            mode='lines+markers+text',
            name=porcentaje,
            text=[f"{v:.2f}%" for v in y],
            textposition=text_positions,
            textfont=dict(size=13, color='black'),
            marker=dict(color=color),
            line=dict(color=color),
            customdata=np.stack([n, total, y], axis=-1),
            hovertemplate=(
                'Periodo: %{x}<br>' +
                porcentaje + ': %{y:.2f}%<br>' +
                'Órdenes con costo: %{customdata[0]}<br>' +
                'Órdenes totales: %{customdata[1]}'
            )
        ))

    # Promedio como línea con hover general (toda la línea muestra el mismo hover)
    for porcentaje, _, color in componentes:
        if porcentaje in columnas_estadistica and porcentaje in completitud_groupby.columns:
            valores = completitud_groupby[porcentaje]
            promedio = np.mean(valores)  # <-- Cambia mediana por promedio
            q1 = np.percentile(valores, 25)
            q3 = np.percentile(valores, 75)
            fig.add_trace(go.Scatter(
                x=periodos,
                y=[promedio]*len(periodos),
                mode='lines+text',
                name=f'Promedio ({porcentaje}) - {promedio:.2f}%',
                line=dict(color=color, dash=dash_estadistica, width=2),
                opacity=1,
                showlegend=True,
                hovertemplate=(
                    f"Promedio {porcentaje}: {promedio:.2f}%<br>"
                    f"IQR: [{q1:.2f}%, {q3:.2f}%]<br>"
                ),
            ))

    fig.update_layout(
        title='Porcentaje de Órdenes con Costo por Componente',
        xaxis_title='Periodo',
        yaxis_title='Porcentaje (%)',
        template='plotly_white',
        height=700,
        width=1000,
        margin=dict(l=20, r=20, t=100, b=20),
    )

    return fig

def show_info_columns(df):
    from graph_hist_utils import streamlit_viz_selector, get_viz_figure
    import numpy as np
    
    # --- Calcula los indicadores ---
    info_df = {}

    # Generales
    info_df['EC'] = len(df['EC'].unique())
    info_df['Proyectos'] = len(df['Proyecto'].unique())
    info_df['Clientes'] = len(df['Cliente'].unique())
    info_df['Unidades'] = len(df['Tracto'].unique())
    info_df['Conductores'] = len(df['Conductor'].unique())
    info_df['No. Rutas'] = len(df['Ruta Ciudades'].unique())
    info_df['No. de Órdenes'] = len(df)
    info_df['Periodo'] = len(df['Periodo'].unique())

    # Costos, CPK y Kms Recorridos
    info_df['Costo Combustible'] = df['Costo Combustible'].sum()
    info_df['Costo Peajes'] = df['Costo Peajes'].sum()
    info_df['Costo Mantenimiento'] = df['Costo Mantenimiento'].sum()
    info_df['Costo Total'] = info_df['Costo Combustible'] + info_df['Costo Peajes'] + info_df['Costo Mantenimiento']
    info_df['kms Totales Recorridos'] = df['kmstotales'].sum()
    info_df['Costo Por Km (CPK)'] = info_df['Costo Total'] / info_df['kms Totales Recorridos'] if info_df['kms Totales Recorridos'] > 0 else 0
    info_df['Costo por Litro'] = df['Costo por litro'].fillna(0).mean() if not df['Costo por litro'].empty else 0


    # Variaciones
    df_for_desc = df.copy()
    df_for_desc = df_for_desc.replace([np.inf, -np.inf], np.nan)

    def q1(series):
        return series.quantile(0.25)
    def q3(series):
        return series.quantile(0.75)

    # Para cada variable, calcula los stats
    def get_stats(col, money=False):
        if col in df_for_desc.columns:
            s = df_for_desc[col].dropna()
            return [
                ("Q1", f"<b>${s.quantile(0.25):,.2f}</b>" if money else f"<b>{s.quantile(0.25):,.2f}</b>"),
                ("Mediana", f"<b>${s.median():,.2f}</b>" if money else f"<b>{s.median():,.2f}</b>"),
                ("Q3", f"<b>${s.quantile(0.75):,.2f}</b>" if money else f"<b>{s.quantile(0.75):,.2f}</b>"),
                ("Mínimo", f"<b>${s.min():,.2f}</b>" if money else f"<b>{s.min():,.2f}</b>"),
                ("Máximo", f"<b>${s.max():,.2f}</b>" if money else f"<b>{s.max():,.2f}</b>"),
            ]
        else:
            return [
                ("Q1", "<b>No disponible</b>"),
                ("Mediana", "<b>No disponible</b>"),
                ("Q3", "<b>No disponible</b>"),
                ("Mínimo", "<b>No disponible</b>"),
                ("Máximo", "<b>No disponible</b>"),
            ]

    # Estructura agrupada en 4 columnas/secciones
    columns_structure = [
        {
            "title": "Generales",
            "content": [
                ("EC distintos", f"<b>{info_df['EC']}</b>"),
                ("Proyectos distintos", f"<b>{info_df['Proyectos']}</b>"),
                ("Clientes distintos", f"<b>{info_df['Clientes']}</b>"),
                ("Unidades distintas", f"<b>{info_df['Unidades']}</b>"),
                ("Conductores distintos", f"<b>{info_df['Conductores']}</b>"),
                ("Rutas distintas", f"<b>{info_df['No. Rutas']}</b>"),
                ("Órdenes totales", f"<b>{info_df['No. de Órdenes']}</b>"),
                ("Periodos distintos", f"<b>{info_df['Periodo']}</b>"),
            ],
        },
        {
            "title": "Costos Kms Totales",
            "content": [
                ("Costo Combustible", f"<b>${info_df['Costo Combustible']:,.2f}</b>"),
                ("Costo Peajes", f"<b>${info_df['Costo Peajes']:,.2f}</b>"),
                ("Costo Mantenimiento", f"<b>${info_df['Costo Mantenimiento']:,.2f}</b>"),
                ("Costo Total", f"<b>${info_df['Costo Total']:,.2f}</b>"),
                ("KMs Totales Recorridos", f"<b>{info_df['kms Totales Recorridos']:,.2f}</b>"),
                ("Media Costo por Litro Orden", f"<b>${info_df['Costo por Litro']:.2f}</b>"),
            ],
        },
        {
            "title": "Costo de Combustible p/ Orden",
            "content": [
                ("Promedio", f"<b>${df_for_desc['Costo Combustible'].mean():,.2f}</b>"),
                ("Desviación Estándar", f"<b>${df_for_desc['Costo Combustible'].std():,.2f}</b>"),
                ("Q1", f"<b>${df_for_desc['Costo Combustible'].quantile(0.25):,.2f}</b>"),
                ("Mediana", f"<b>${df_for_desc['Costo Combustible'].median():,.2f}</b>"),
                ("Q3", f"<b>${df_for_desc['Costo Combustible'].quantile(0.75):,.2f}</b>"),
                ("Mínimo", f"<b>${df_for_desc['Costo Combustible'].min():,.2f}</b>"),
                ("Máximo", f"<b>${df_for_desc['Costo Combustible'].max():,.2f}</b>"),
            ],
        },
        {
            "title": "Costo de Peajes p/ Orden",
            "content": [
                ("Promedio", f"<b>${df_for_desc['Costo Peajes'].mean():,.2f}</b>"),
                ("Desviación Estándar", f"<b>${df_for_desc['Costo Peajes'].std():,.2f}</b>"),
                ("Q1", f"<b>${df_for_desc['Costo Peajes'].quantile(0.25):,.2f}</b>"),
                ("Mediana", f"<b>${df_for_desc['Costo Peajes'].median():,.2f}</b>"),
                ("Q3", f"<b>${df_for_desc['Costo Peajes'].quantile(0.75):,.2f}</b>"),
                ("Mínimo", f"<b>${df_for_desc['Costo Peajes'].min():,.2f}</b>"),
                ("Máximo", f"<b>${df_for_desc['Costo Peajes'].max():,.2f}</b>"),
            ],
        },
        {
            "title": "Costo de Mantenimiento p/ Orden",
            "content": [
                ("Promedio", f"<b>${df_for_desc['Costo Mantenimiento'].mean():,.2f}</b>"),
                ("Desviación Estándar", f"<b>${df_for_desc['Costo Mantenimiento'].std():,.2f}</b>"),
                ("Q1", f"<b>${df_for_desc['Costo Mantenimiento'].quantile(0.25):,.2f}</b>"),
                ("Mediana", f"<b>${df_for_desc['Costo Mantenimiento'].median():,.2f}</b>"),
                ("Q3", f"<b>${df_for_desc['Costo Mantenimiento'].quantile(0.75):,.2f}</b>"),
                ("Mínimo", f"<b>${df_for_desc['Costo Mantenimiento'].min():,.2f}</b>"),
                ("Máximo", f"<b>${df_for_desc['Costo Mantenimiento'].max():,.2f}</b>"),
            ],
        },
        {
            "title": "Kms Recorridos p/ Orden", 
            "content": [
                ("Promedio", f"<b>{df_for_desc['kmstotales'].mean():,.2f}</b> km"),
                ("Desviación Estándar", f"<b>{df_for_desc['kmstotales'].std():,.2f}</b> km"),
                ("Q1", f"<b>{df_for_desc['kmstotales'].quantile(0.25):,.2f}</b> km"),
                ("Mediana", f"<b>{df_for_desc['kmstotales'].median():,.2f}</b> km"),
                ("Q3", f"<b>{df_for_desc['kmstotales'].quantile(0.75):,.2f}</b> km"),
                ("Mínimo", f"<b>{df_for_desc['kmstotales'].min():,.2f}</b> km"),
                ("Máximo", f"<b>{df_for_desc['kmstotales'].max():,.2f}</b> km"),
            ],
        }
        ]

    # Calcula el máximo de filas para dar el mismo min-height
    max_items = max(len(col["content"]) for col in columns_structure)
    row_height = 28  # px, ajusta si quieres más separacion vertical
    min_height = row_height * (max_items + 2) + 40  # +2 por título y buffer

    st.markdown(f"""
    <style>
    .mega-flex {{
        display: flex;
        flex-direction: row;
        gap: 0px;
        margin-bottom: 50px;
        margin-top: 10px;
        min-height: {min_height}px;
    }}
    .mega-flex-col {{
        flex: 1 1 0%;
        padding: 10px 18px 30px 18px;
        box-sizing: border-box;
        min-height: {min_height}px;
        display: flex;
        flex-direction: column;
        justify-content: flex-start;
    }}
    .mega-flex-col:not(:last-child) {{
        border-right: 2px solid #e3e3e3;
    }}
    .mega-title {{
        font-weight: bold;
        font-size: 15px;
        margin-bottom: 12px;
        margin-top: 2px;
    }}
    .mega-item {{
        margin-bottom: 6px;
        font-size: 15px;
        display: block;
    }}
    .mega-item-label {{
        font-weight: bold;
        margin-top: 12px;
        display: block;
    }}
    .mega-item-value {{
        font-weight: bold;
        margin-left: 8px;
        color: #173A5E;
    }}
    @media (max-width: 950px) {{
        .mega-flex {{ flex-direction: column; }}
        .mega-flex-col {{ border-right: none !important; border-bottom: 2px solid #e3e3e3; }}
        .mega-flex-col:last-child {{ border-bottom: none !important; }}
    }}
    </style>
    """, unsafe_allow_html=True)

    
    html = '<div class="mega-flex">'
    for col in columns_structure:
        html += '<div class="mega-flex-col">'
        html += f'<div class="mega-title">{col["title"]}</div>'
        last_label = None
        for k, v in col["content"]:
            if v == "":
                html += f'<span class="mega-item-label">{k}:</span>'
            else:
                html += f'<span class="mega-item">{k}: <span class="mega-item-value">{v}</span></span>'
        html += '</div>'
    html += '</div>'
    st.markdown(html, unsafe_allow_html=True)
        

In [4]:
%%writefile historial_cargas.py

# Este archivo define funciones para analizar el historial de cargas de combustible entre eventos para cada tracto.
# Permite calcular métricas operativas entre recargas de combustible.

# Función: historial_entre_cargas
# - Procesa el DataFrame de órdenes para cada tracto, identificando los periodos entre cargas de combustible.
# - Calcula métricas como kms recorridos, costos, rendimiento, CPK y otros indicadores entre cargas.
# - Devuelve dos DataFrames: uno detallado por evento de carga y otro agrupado por tracto.
# - Es fundamental para analizar el desempeño operativo y los costos entre recargas.


def historial_entre_cargas(df):

    import pandas as pd
    import numpy as np
    from numpy import mean


    unidades = df['Tracto'].unique()

    df_p = df[(df['Periodo'] >= '2025-01') & (df['Periodo'] < '2025-03')]
    df_p = df.copy()

    historial_cargas = []

    for ud in unidades:
        df_ud = df_p[df_p['Tracto'] == ud].sort_values('Inicio de la Orden', ascending=True)
        cont_kms = 0
        cont_cargas = 0
        viajes = 0
        fecha_ant_carga = np.nan
        no_rutas_distintas = set()
        no_proyectos_distintos = set()
        mean_kms_l = []

        costo_peajes_entre_cargas = 0
        costo_mant_entre_cargas = 0

        for i in range(len(df_ud)):
            if df_ud['Orden con Costo de Combustible'].iloc[i]:
                no_rutas_distintas.add(df_ud['Ruta Ciudades'].iloc[i])
                no_proyectos_distintos.add(df_ud['Proyecto'].iloc[i])
                mean_kms_l.append(df_ud['kmstotales'].iloc[i])
                cont_kms += df_ud['kmstotales'].iloc[i]
                cont_cargas += 1
                fecha_carga = df_ud['Cierre de la Orden'].iloc[i]
                rutas = len(no_rutas_distintas)
                proyectos = len(no_proyectos_distintos)
                mean_kms = sum(mean_kms_l) / len(mean_kms_l) if mean_kms_l else 0
                litros_carga = df_ud['Litros'].iloc[i]
                costo_plitro = df_ud['Costo por litro'].iloc[i]
                costo_carga = df_ud['Costo Combustible'].iloc[i]

                costo_peajes_entre_cargas += df_ud['Costo Peajes'].iloc[i]
                costo_mant_entre_cargas += df_ud['Costo Mantenimiento'].iloc[i] 

                costo_total = costo_carga + costo_peajes_entre_cargas + costo_mant_entre_cargas
                
                # --- Corrección para evitar división por cero ---
                if litros_carga != 0:
                    rendimiento = cont_kms / litros_carga
                else:
                    rendimiento = np.nan
                if cont_kms != 0:
                    cpk = costo_carga / cont_kms
                    cpk_peajes = costo_peajes_entre_cargas / cont_kms
                    cpk_mant = costo_mant_entre_cargas / cont_kms
                else:
                    cpk = np.nan
                    cpk_peajes = np.nan
                    cpk_mant = np.nan

                periodo = df_ud['Periodo'].iloc[i]
                kms_por_dia = cont_kms / (fecha_carga - fecha_ant_carga).days if pd.notna(fecha_ant_carga) and (fecha_carga - fecha_ant_carga).days != 0 else 0
                historial_cargas.append([
                    periodo, ud, cont_cargas, fecha_carga, fecha_ant_carga, litros_carga,costo_plitro,rendimiento, costo_carga, costo_peajes_entre_cargas, costo_mant_entre_cargas,
                    costo_total, cpk,cpk_peajes,cpk_mant, cont_kms, viajes, rutas, proyectos, mean_kms, kms_por_dia
                ])
                fecha_ant_carga = fecha_carga
                cont_kms = 0
                viajes = 0
                no_rutas_distintas = set()
                no_proyectos_distintos = set()
                mean_kms_l = []

                costo_peajes_entre_cargas = 0
                costo_mant_entre_cargas = 0
            else:
                cont_kms += df_ud['kmstotales'].iloc[i]
                no_rutas_distintas.add(df_ud['Ruta Ciudades'].iloc[i])
                no_proyectos_distintos.add(df_ud['Proyecto'].iloc[i])
                mean_kms_l.append(df_ud['kmstotales'].iloc[i])

                costo_peajes_entre_cargas += df_ud['Costo Peajes'].iloc[i]
                costo_mant_entre_cargas += df_ud['Costo Mantenimiento'].iloc[i]

            viajes += 1

    historial_cargas = pd.DataFrame(
        historial_cargas, 
        columns=[
            'Periodo',
            'Tracto',
            'No. de Carga Combustible',
            'Fecha Orden de Ant. Carga',
            'Fecha Orden de Carga',
            'Litros Combustible Cargados',
            'Costo por Litro',
            'Rendimiento Kms/Litro',
            'Costo de Combustible',
            'Costo de Peajes',
            'Costo de Mantenimiento',
            'Costo Total',            
            'CPK Combustible',
            'CPK Peajes',
            'CPK Mantenimiento',
            'KMs Recorridos desde Última Carga',
            'Viajes entre Cargas',
            'Rutas Distintas',
            'Proyectos Distintos',
            'Promedio Kms por Viaje',
            'Kms Recorridos por Día'
        ]
    )

    historial_cargas['Tiempo entre Cargas'] = (
        historial_cargas['Fecha Orden de Carga'] - historial_cargas['Fecha Orden de Ant. Carga']
    ).dt.total_seconds() / (24 * 3600)

    # Limpiar inf y NaN en filas
    historial_cargas = historial_cargas[
        ~historial_cargas['Tiempo entre Cargas'].isin([np.inf, -np.inf]) &
        ~historial_cargas['Tiempo entre Cargas'].isna()
    ]

    historial_cargas['Kms Totales'] = historial_cargas['KMs Recorridos desde Última Carga'].fillna(0)
    historial_cargas['No. Viajes'] = historial_cargas['Viajes entre Cargas'].fillna(0)

    hist_cargas_grouped = historial_cargas.groupby(['Tracto']).agg({
        'No. de Carga Combustible': 'median',
        'Tiempo entre Cargas': 'mean',
        'KMs Recorridos desde Última Carga': 'mean',
        'Litros Combustible Cargados': 'mean',
        'Costo por Litro': 'mean',
        'Rendimiento Kms/Litro': 'mean',
        'Costo de Combustible': 'mean',
        'Costo de Peajes': 'mean',
        'Costo de Mantenimiento': 'mean',
        'Costo Total': 'mean',
        'CPK Combustible': 'mean',
        'CPK Peajes': 'mean',
        'CPK Mantenimiento': 'mean',
        'Viajes entre Cargas': 'median',
        'Rutas Distintas': 'median',
        'Proyectos Distintos': 'median',
        'Promedio Kms por Viaje': 'mean',
        'Kms Recorridos por Día': 'mean',
        'Kms Totales': 'sum',
        'No. Viajes': 'sum'
    }).reset_index()

    hist_cargas_grouped.set_index('Tracto', inplace=True)

    return historial_cargas, hist_cargas_grouped

In [5]:
%%writefile calculos_cpk.py

# Este archivo contiene funciones para calcular y visualizar el Costo Por Kilómetro (CPK) desglosado por componentes.

# Función: agrupar_componentes_cpk
# - Agrupa y calcula el CPK de combustible, peajes y mantenimiento bajo diferentes criterios (todas las órdenes, solo con costo, solo con componente, entre cargas).
# - Devuelve un DataFrame con todos los indicadores necesarios para el análisis comparativo.

# Función: plot_cpk_barras_comparativo
# - Genera un gráfico de barras apiladas con Plotly para comparar el CPK por periodo y por criterio de agrupación.
# - Usa colores y posiciones diferenciadas para cada grupo y componente.
# - Permite visualizar rápidamente cómo varía el CPK según el método de cálculo.

# Función: cpk_desglosado
# - Orquesta el cálculo y visualización del CPK desglosado.
# - Llama a las funciones anteriores y muestra los resultados en la app Streamlit.
# - Permite al usuario comparar visualmente los componentes de costo y su evolución.

def agrupar_componentes_cpk(df,historial_cargas):

    import pandas as pd

    df_agg_all = df.groupby('Periodo').agg({'Costo Combustible':'sum', 'Costo Peajes':'sum', 'Costo Mantenimiento':'sum', 'kmstotales':'sum','Litros':'sum'}).sort_values('Periodo', ascending=True)
    df_agg_all['CPK Combustible (Todas las Órdenes)'] = df_agg_all['Costo Combustible'] / df_agg_all['kmstotales']
    df_agg_all['CPK Peajes (Todas las Órdenes)'] = df_agg_all['Costo Peajes'] / df_agg_all['kmstotales']
    df_agg_all['CPK Mantenimiento (Todas las Órdenes)'] = df_agg_all['Costo Mantenimiento'] / df_agg_all['kmstotales']
    df_agg_all['Rendimiento Kms/Litro (Todas las Órdenes)'] = df_agg_all['kmstotales'] / df_agg_all['Litros']
    df_agg_all['No. Órdenes Consideradas (Todas las Órdenes)'] = df.groupby('Periodo').size()
    df_agg_all['Costo por Litro (Todas las Órdenes)'] = df_agg_all['Costo Combustible'] / df_agg_all['Litros']



    df_agg_all['Costo Combustible (Todas las Órdenes)'] = df_agg_all['Costo Combustible'].fillna(0)
    df_agg_all['Litros (Todas las Órdenes)'] = df_agg_all['Litros'].fillna(0)

    df_filtered = df[df['Costo Total'] > 0].copy()
    df_agg_filtered = df_filtered.groupby('Periodo').agg({'Costo Combustible':'sum', 'Costo Peajes':'sum', 'Costo Mantenimiento':'sum', 'kmstotales':'sum','Litros':'sum'}).sort_values('Periodo', ascending=True)
    df_agg_filtered['CPK Combustible (Órdenes con costo)'] = df_agg_filtered['Costo Combustible'] / df_agg_filtered['kmstotales']
    df_agg_filtered['CPK Peajes (Órdenes con costo)'] = df_agg_filtered['Costo Peajes'] / df_agg_filtered['kmstotales']
    df_agg_filtered['CPK Mantenimiento (Órdenes con costo)'] = df_agg_filtered['Costo Mantenimiento'] / df_agg_filtered['kmstotales']
    df_agg_filtered['Rendimiento Kms/Litro (Órdenes con costo)'] = df_agg_filtered['kmstotales'] / df_agg_filtered['Litros']
    df_agg_filtered['No. Órdenes Consideradas (Órdenes con costo)'] = df_filtered.groupby('Periodo').size()
    df_agg_filtered['Costo por Litro (Órdenes con costo)'] = df_agg_filtered['Costo Combustible'] / df_agg_filtered['Litros']

    df_componente_comb = df[df['Orden con Costo de Combustible'] == True].copy()
    df_componente_peajes = df[df['Orden con Costo de Peajes'] == True].copy()
    df_componente_mant = df[df['Orden con Costo de Mantenimiento'] == True].copy()

    df_agg_componente_comb = df_componente_comb.groupby('Periodo').agg({'Costo Combustible':'sum', 'kmstotales':'sum','Litros':'sum'}).sort_values('Periodo', ascending=True)
    df_agg_componente_comb['CPK Combustible (Órdenes con Componente)'] = df_agg_componente_comb['Costo Combustible'] / df_agg_componente_comb['kmstotales']
    df_agg_componente_comb['Rendimiento Kms/Litro (Órdenes con Componente)'] = df_agg_componente_comb['kmstotales'] / df_agg_componente_comb['Litros']
    df_agg_componente_comb['No. Órdenes Consideradas (Órdenes con Combustible)'] = df_componente_comb.groupby('Periodo').size()
    df_agg_componente_comb['Costo por Litro (Órdenes con Componente)'] = df_agg_componente_comb['Costo Combustible'] / df_agg_componente_comb['Litros']

    df_agg_componente_peajes = df_componente_peajes.groupby('Periodo')[['Costo Peajes', 'kmstotales']].sum().sort_values('Periodo', ascending=True)
    df_agg_componente_peajes['CPK Peajes (Órdenes con Componente)'] = df_agg_componente_peajes['Costo Peajes'] / df_agg_componente_peajes['kmstotales']
    df_agg_componente_peajes['No. Órdenes Consideradas (Órdenes con Peaje)'] = df_componente_peajes.groupby('Periodo').size()

    df_agg_componente_mant = df_componente_mant.groupby('Periodo')[['Costo Mantenimiento', 'kmstotales']].sum().sort_values('Periodo', ascending=True)
    df_agg_componente_mant['CPK Mantenimiento (Órdenes con Componente)'] = df_agg_componente_mant['Costo Mantenimiento'] / df_agg_componente_mant['kmstotales']
    df_agg_componente_mant['No. Órdenes Consideradas (Órdenes con Mantenimiento)'] = df_componente_mant.groupby('Periodo').size()

    hist_cargas = historial_cargas[historial_cargas['Periodo'].isin(df['Periodo'].unique())].copy()
    df_cargaxcarga = hist_cargas.groupby('Periodo').agg({'Costo de Combustible': 'sum','KMs Recorridos desde Última Carga': 'sum','Litros Combustible Cargados': 'sum'})
    df_cargaxcarga['CPK Combustible (Entre Cargas)'] = df_cargaxcarga['Costo de Combustible'] / df_cargaxcarga['KMs Recorridos desde Última Carga']
    df_cargaxcarga['Rendimiento Kms/Litro (Entre Cargas)'] = df_cargaxcarga['KMs Recorridos desde Última Carga'] / df_cargaxcarga['Litros Combustible Cargados']
    df_cargaxcarga['No. Cargas con Combustible'] = hist_cargas.groupby('Periodo').size()
    df_cargaxcarga['Costo por Litro (Entre Cargas)'] = df_cargaxcarga['Costo de Combustible'] / df_cargaxcarga['Litros Combustible Cargados']

    df_cargaxcarga['CPK Peajes (Entre Cargas)'] = hist_cargas.groupby('Periodo')['Costo de Peajes'].sum() / df_cargaxcarga['KMs Recorridos desde Última Carga']
    df_cargaxcarga['No. Cargas con Peajes'] = hist_cargas.groupby('Periodo')['Costo de Peajes'].apply(lambda x: (x > 0).sum())

    df_cargaxcarga['CPK Mantenimiento (Entre Cargas)'] = hist_cargas.groupby('Periodo')['Costo de Mantenimiento'].sum() / df_cargaxcarga['KMs Recorridos desde Última Carga']    
    df_cargaxcarga['No. Cargas con Mantenimiento'] = hist_cargas.groupby('Periodo')['Costo de Mantenimiento'].apply(lambda x: (x > 0).sum())
   

    df_all = pd.concat([df_agg_all[['CPK Combustible (Todas las Órdenes)', 'CPK Peajes (Todas las Órdenes)', 'CPK Mantenimiento (Todas las Órdenes)',
                        'Rendimiento Kms/Litro (Todas las Órdenes)', 'No. Órdenes Consideradas (Todas las Órdenes)','Costo por Litro (Todas las Órdenes)']],
                        
                        df_agg_filtered[['CPK Combustible (Órdenes con costo)', 'CPK Peajes (Órdenes con costo)', 'CPK Mantenimiento (Órdenes con costo)',
                        'Rendimiento Kms/Litro (Órdenes con costo)', 'No. Órdenes Consideradas (Órdenes con costo)', 'Costo por Litro (Órdenes con costo)']],
                        
                        df_agg_componente_comb[['CPK Combustible (Órdenes con Componente)', 'Rendimiento Kms/Litro (Órdenes con Componente)',
                         'No. Órdenes Consideradas (Órdenes con Combustible)', 'Costo por Litro (Órdenes con Componente)']],

                        df_agg_componente_peajes[['CPK Peajes (Órdenes con Componente)', 'No. Órdenes Consideradas (Órdenes con Peaje)']],
                        df_agg_componente_mant[['CPK Mantenimiento (Órdenes con Componente)', 'No. Órdenes Consideradas (Órdenes con Mantenimiento)']], 
                        
                        df_cargaxcarga[['CPK Combustible (Entre Cargas)', 'Rendimiento Kms/Litro (Entre Cargas)', 'No. Cargas con Combustible','Costo por Litro (Entre Cargas)',
                        'CPK Peajes (Entre Cargas)', 'No. Cargas con Peajes', 'CPK Mantenimiento (Entre Cargas)', 'No. Cargas con Mantenimiento']]
                        ], axis=1)

    df_all.fillna(0, inplace=True)

    return df_all

def plot_cpk_barras_comparativo(
    df_all, 
    componentes=['Combustible', 'Peajes', 'Mantenimiento'], 
    grupos=['Todas las Órdenes', 'Órdenes con costo', 'Órdenes con Componente', 'Entre Cargas'],
    width=1200, 
    height=600
):
    import plotly.graph_objects as go
    import numpy as np

    # Colores sólidos únicos para cada grupo+componente
    COLOR_MAP = {
        ('Todas las Órdenes', 'Combustible'): "#A3BFFA",   # Azul pastel (más claro)
        ('Órdenes con costo', 'Combustible'): "#5086F2",   # Azul claro
        ('Órdenes con Componente', 'Combustible'): "#4361EE", # Azul medio
        ('Entre Cargas', 'Combustible'): "#233ED9",        # Azul fuerte (más oscuro)
        ('Todas las Órdenes', 'Peajes'): "#FFF9DB",        # Amarillo pastel (más claro)
        ('Órdenes con costo', 'Peajes'): "#FFF3BF",        # Amarillo claro
        ('Órdenes con Componente', 'Peajes'): "#F9E79F",   # Amarillo medio
        ('Entre Cargas', 'Peajes'): "#F2CD5E",             # Amarillo fuerte (más oscuro)
        ('Todas las Órdenes', 'Mantenimiento'): "#C9ADA7", # Gris pastel (más claro)
        ('Órdenes con costo', 'Mantenimiento'): "#9A8C98", # Gris claro
        ('Órdenes con Componente', 'Mantenimiento'): "#4A4E69", # Gris medio
        ('Entre Cargas', 'Mantenimiento'): "#22223B",      # Gris oscuro (más oscuro)
    }

    col_map = {
        'Todas las Órdenes': lambda comp: f'CPK {comp} (Todas las Órdenes)',
        'Órdenes con costo': lambda comp: f'CPK {comp} (Órdenes con costo)',
        'Órdenes con Componente': lambda comp: f'CPK {comp} (Órdenes con Componente)',
        'Entre Cargas': lambda comp: f'CPK {comp} (Entre Cargas)',
    }
    pos_map = {
        'Todas las Órdenes': -0.3,
        'Órdenes con costo': -0.1,
        'Órdenes con Componente': 0.1,
        'Entre Cargas': 0.3,
    }

    periodos = df_all.index.astype(str)
    n = len(periodos)
    ind = np.arange(n)
    bar_width = 0.18

    fig = go.Figure()

    ordenes_col_map = {
        'Todas las Órdenes': 'No. Órdenes Consideradas (Todas las Órdenes)',
        'Órdenes con costo': 'No. Órdenes Consideradas (Órdenes con costo)',
        'Órdenes con Componente': {
            'Combustible': 'No. Órdenes Consideradas (Órdenes con Combustible)',
            'Peajes': 'No. Órdenes Consideradas (Órdenes con Peaje)',
            'Mantenimiento': 'No. Órdenes Consideradas (Órdenes con Mantenimiento)'
        },
        'Entre Cargas': {
            'Combustible': 'No. Cargas con Combustible',
            'Peajes': 'No. Cargas con Peajes',
            'Mantenimiento': 'No. Cargas con Mantenimiento'
        }
    }

    for grupo in grupos:
        comps_presentes = []
        custom_cols = []
        for comp in componentes:
            colname = col_map[grupo](comp)
            if colname in df_all.columns:
                comps_presentes.append(comp)
                custom_cols.append(df_all[colname])
        for comp in comps_presentes:
            col = col_map[grupo](comp)
            y_vals = df_all[col]
            base = 0
            if comp == 'Peajes' and f'CPK Combustible ({grupo})' in df_all.columns:
                base = df_all[col_map[grupo]('Combustible')]
            elif comp == 'Mantenimiento' and all(f'CPK {c} ({grupo})' in df_all.columns for c in ['Combustible', 'Peajes']):
                base = df_all[col_map[grupo]('Combustible')] + df_all[col_map[grupo]('Peajes')]

            suma_total = np.sum(custom_cols, axis=0) if custom_cols else np.zeros_like(y_vals)
            if isinstance(ordenes_col_map[grupo], dict):
                ordenes_col_name = ordenes_col_map[grupo].get(comp)
            else:
                ordenes_col_name = ordenes_col_map[grupo]
            n_ordenes_col = df_all[ordenes_col_name] if ordenes_col_name in df_all.columns else np.full_like(y_vals, np.nan)
            
            customdata = np.stack(
                [periodos] + [c.values for c in custom_cols] + [suma_total, n_ordenes_col.values],
                axis=-1
            )
            hover_lines = [f"<b>{grupo}</b><br>", "Periodo: %{customdata[0]}<br>"]
            for idx, c_label in enumerate(comps_presentes):
                if c_label == comp:
                    hover_lines.append(f"<b>{c_label}: $%{{customdata[{idx+1}]:,.4f}}</b><br>")
                else:
                    hover_lines.append(f"{c_label}: $%{{customdata[{idx+1}]:,.4f}}<br>")
            hover_lines.append(f"Acumulado total: $%{{customdata[{len(comps_presentes)+1}]:,.4f}}<br>")
            hover_lines.append(f"Órdenes consideradas: %{{customdata[{len(comps_presentes)+2}]}}<br>")
            hover_lines.append("<extra></extra>")
            hovertemplate = ''.join(hover_lines)

            # Color sólido único para cada grupo+componente
            color = COLOR_MAP.get((grupo, comp), "#888888")

            fig.add_trace(go.Bar(
                x=ind + pos_map[grupo],
                y=y_vals,
                name=f'{comp} ({grupo})',
                marker_color=color,
                width=bar_width,
                offsetgroup=grupo,
                legendgroup=f"{grupo}-{comp}",
                showlegend=True,
                base=base if comp != 'Combustible' else None,
                opacity=1.0,  # Sin opacidad
                customdata=customdata,
                hovertemplate=hovertemplate
            ))

            for i, val in enumerate(suma_total):
                if comp == comps_presentes[-1]:
                    fig.add_annotation(
                        x=ind[i] + pos_map[grupo],
                        y=val,
                        text=f" ${val:.2f}",
                        showarrow=False,
                        yshift=12.5,
                        font=dict(size=13, color="#222"),
                        bgcolor="rgba(255,255,255,0.85)",
                        align="center"
                    )

    fig.update_layout(
        barmode='stack',
        title='CPK por Periodo y Componentes',
        xaxis=dict(
            title='Periodo',
            tickvals=ind,
            ticktext=periodos
        ),
        yaxis_title='CPK ($/km)',
        template='plotly_white',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.08,
            xanchor="center",
            x=0.5,
            font=dict(size=13)
        ),
        legend_itemclick=False,
        legend_itemdoubleclick=False,
        margin=dict(l=50, r=30, t=200, b=50),
        width=width,
        height=height
    )
    
    return fig

def cpk_desglosado(df,historial_cargas):

    from calculos_cpk import agrupar_componentes_cpk, plot_cpk_barras_comparativo
    from comparar_comp_utils import comparar_componentes_cpk, construir_df_cpk_periodo
    import streamlit as st
    import pandas as pd

    df_all = agrupar_componentes_cpk(df, historial_cargas)
    fig = plot_cpk_barras_comparativo(
        df_all,
        componentes=['Combustible', 'Peajes'],
        grupos=['Todas las Órdenes', 'Órdenes con costo', 'Órdenes con Componente', 'Entre Cargas'],
        width=1200,
        height=800
    )
    st.plotly_chart(fig, use_container_width=True)
    
    df_cpk_periodo = construir_df_cpk_periodo(df_all, grupos=['todas', 'costo', 'componente', 'cargas'])

    st.subheader("Comparación de Componentes e Indicadores de Rendimiento por Periodo")

    st.info(""" Selecciona un periodo para comparar los indicadores de rendimiento (CPK, Rendimiento Kms/Litro, Costo por Litro) entre los componentes (Combustible, Peajes).""")

    df['Periodo'] = pd.PeriodIndex(df['Periodo'], freq='M')

    # selecciona rango de periodos
    periodo_inicio = df['Periodo'].min()
    periodo_fin = df['Periodo'].max()

    periodos_opciones = sorted(df['Periodo'].unique())

    col1, col2 = st.columns([2, 3])

    with col1:
        periodo_seleccionado = st.select_slider(
            "Selecciona el periodo para comparar CPK",
            options=periodos_opciones,
            value=(periodos_opciones[0], periodos_opciones[-1]),
            format_func=lambda x: x.strftime('%b %Y')
        )

    periodo_strs = [str(p) for p in periodo_seleccionado]

    st.markdown(
        f"""
        <h4 style='text-align: center;'>
            Comparación de CPK por Componente entre Periodos e Indicadores de Rendimiento: {periodo_strs[0]} y {periodo_strs[1]}
        </h4>
        """,
        unsafe_allow_html=True
    )

    # Lista de variables disponibles
    variables_cpk = [
        'CPK Peajes',
        'CPK Combustible',
        'Rendimiento Kms/Litro',
        'Costo por Litro',
    ]

    # Multiselect con máximo 4 opciones

    col1, col2 = st.columns([2,3])

    with col1:

        seleccionadas = st.multiselect(
            "Selecciona hasta 4 variables para comparar",
            options=variables_cpk,
            default=variables_cpk[:2],
            max_selections=3  # Streamlit >= 1.27
        )

    if len(seleccionadas) == 0:
        st.info("Selecciona al menos una variable para mostrar los gráficos.")
    else:
        cols = st.columns(len(seleccionadas))
        for i, variable in enumerate(seleccionadas):
            with cols[i]:
                fig = comparar_componentes_cpk(
                    df_cpk_periodo,
                    variable=variable,
                    periodos=periodo_strs,
                    width=600,
                    height=800,
                    es_dinero = variable in ['CPK Peajes', 'CPK Combustible', 'Costo por Litro']
 
                )
                st.plotly_chart(fig, use_container_width=True)
        

In [6]:
%%writefile graph_hist_utils.py

# Este archivo contiene utilidades para seleccionar y graficar columnas numéricas en Streamlit.

# Función: streamlit_viz_selector
# - Permite al usuario seleccionar una columna numérica y el tipo de gráfico (barras, boxplot, pastel) desde la interfaz Streamlit.
# - Devuelve la columna y el tipo de gráfico seleccionados.

# Función: get_viz_figure
# - Genera el gráfico correspondiente (barras, boxplot o pastel) para la columna seleccionada.
# - Calcula y muestra estadísticas descriptivas (media, mediana, cuartiles, etc.) en el gráfico.
# - Facilita la exploración visual de la distribución de los datos.

import streamlit as st
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

def streamlit_viz_selector(df,idx = 0, key=""):
    # Detecta columnas numéricas completas (sin strings ni NaN), omite 'Tracto'
    num_cols = [
        col for col in df.columns
        if pd.api.types.is_numeric_dtype(df[col]) and df[col].notna().all() and col not in ["Tracto", "No. Orden", "No. Viajes","Orden con Costo de Combustible"
                                                                                            , "Orden con Costo de Peajes", "Orden con Costo de Mantenimiento"]
    ]

    if not num_cols:
        st.warning("No hay columnas numéricas completas en el DataFrame.")
        return None, None, None

    # Dropdown para elegir UNA columna numérica
    seleccionada = st.selectbox(
        "Selecciona una columna numérica para analizar:",
        options=num_cols,
        index=idx if num_cols else None
        , key=f"col_select_{key}"
    )

    if not seleccionada:
        st.info("Selecciona una columna para continuar.")
        return None, None, None

    # Dropdown para tipo de gráfico
    tipo_grafico = st.selectbox(
        "Selecciona el tipo de gráfico:",
        options=["Barras", "Boxplot", "Pastel"]
        , key=f"tipo_grafico_{key}"
    )

    return seleccionada,tipo_grafico

def get_viz_figure(df, seleccionada, tipo_grafico, width=600, height=500, bar_width=0.25):
    import numpy as np
    import plotly.graph_objects as go
    import pandas as pd
    from plotly.subplots import make_subplots
    import streamlit as st

    col = seleccionada
    data = df[col].dropna()
    if data.empty:
        return None

    def fmt_num(n):
        return f"{n:,.2f}"

    mediana = np.median(data)
    minimo = np.min(data)
    maximo = np.max(data)
    promedio = np.mean(data)
    std = np.std(data)
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)

    stats_text = (
        f"<span style='font-size:22px;'><b>{col}</b></span><br><br>"
        f"<span style='font-size:17px;'>"
        f"<b>Promedio:</b> {fmt_num(promedio)}<br>"
        f"<b>Desv. estándar:</b> {fmt_num(std)}<br>"
        f"<b>Q1:</b> {fmt_num(q1)}<br>"
        f"<b>Mediana:</b> {fmt_num(mediana)}<br>"
        f"<b>Q3:</b> {fmt_num(q3)}<br>"
        f"<b>Mínimo:</b> {fmt_num(minimo)}<br>"
        f"<b>Máximo:</b> {fmt_num(maximo)}</span>"
    )

    if tipo_grafico == "Barras":
        fig = go.Figure()
        x_labels = ["", col, ""]
        y_vals = [None, mediana, None]
        bar_widths = [0, bar_width, 0]
        fig.add_trace(go.Bar(
            x=x_labels,
            y=y_vals,
            name="Mediana",
            marker_color="#4361EE",
            width=bar_widths,
            error_y=dict(
                type='data',
                symmetric=False,
                array=[0, maximo - mediana, 0],
                arrayminus=[0, mediana - minimo, 0],
                color="black",
                thickness=2.5,
                width=10
            ),
        ))
        fig.update_layout(
            yaxis_title=col,
            xaxis_title="",
            template="plotly_white",
            width=width,
            height=height,
            margin=dict(l=20, t=80, r=20, b=40),
            xaxis=dict(
                tickmode='array',
                tickvals=[0, 1, 2],
                ticktext=["", col, ""],
                showgrid=False,
                showticklabels=False
            ),
        )
        fig.add_annotation(
            text=stats_text,
            xref="paper", yref="paper",
            x=0.25, y=0.5,
            showarrow=False,
            align="center",
            bordercolor="#ccc",
            borderwidth=1,
            borderpad=16,
            bgcolor="rgba(255,255,255,0.97)",
            font=dict(size=20, color="#222"),
            xanchor="center", yanchor="middle"
        )
        return fig

    elif tipo_grafico == "Boxplot":
        # El boxplot debe estar en el centro del espacio 2 y 3 (x=1.5)
        fig = go.Figure()
        fig.add_trace(go.Box(
            x=[1.5] * len(data),  # posición central entre 1 y 2
            y=data,
            boxpoints="outliers",
            marker_color="#4361EE",
            width=bar_width,
            name=col,
            fillcolor="rgba(67,97,238,0.15)",
            line=dict(width=2, color="#4361EE"),
            showlegend=False
        ))
        fig.update_layout(
            margin=dict(l=20, t=80, r=20, b=40),
            width=width,
            height=height,
            xaxis=dict(
                range=[-0.2, 2.2],
                tickmode='array',
                tickvals=[0, 1, 1.5, 2],
                ticktext=["", "", col, ""],
                showgrid=False,
                showticklabels=True
            ),
            yaxis_title=col,
            template="plotly_white"
        )
        fig.add_annotation(
            text=stats_text,
            xref="paper", yref="paper",
            x=0.25, y=0.5,
            showarrow=False,
            align="center",
            bordercolor="#ccc",
            borderwidth=1,
            borderpad=16,
            bgcolor="rgba(255,255,255,0.97)",
            font=dict(size=20, color="#222"),
            xanchor="center", yanchor="middle"
        )
        return fig

    elif tipo_grafico == "Pastel":
        if minimo == maximo:
            # Todos los valores son iguales, no se puede hacer bins
            st.warning("No se puede graficar pastel: todos los valores son iguales.")
            return None
        bins = np.linspace(minimo, maximo, 11)
        labels = [f"{fmt_num(bins[i])} - {fmt_num(bins[i+1])}" for i in range(len(bins)-1)]
        categorias = pd.cut(data, bins=bins, labels=labels, include_lowest=True)
        conteo = categorias.value_counts().sort_index()
        legend_labels = [l for l, v in zip(labels, conteo.values) if v > 0]
        legend_vals = [v for v in conteo.values if v > 0]

        fig = make_subplots(
            rows=1, cols=2,
            column_widths=[0.75, 0.25],
            specs=[[{"type": "domain"}, {"type": "domain"}]]
        )
        fig.add_trace(
            go.Pie(
                labels=legend_labels,
                values=legend_vals,
                hole=0.2,
                textinfo='percent',
                showlegend=True,
                marker=dict(line=dict(color='#fff', width=1.5)),
            ),
            row=1, col=1
        )
        fig.update_layout(
            width=width,
            height=height,
            margin=dict(l=20, t=80, r=20, b=40),
            legend=dict(
                x=0.8,         # fuera del área de la gráfica
                y=0.8,            # arriba
                xanchor="left", # ancla a la izquierda de la caja de leyenda
                yanchor="top",  # ancla arriba
                font=dict(size=15),
                bgcolor="rgba(255,255,255,0.95)",
                bordercolor="#ccc",
                borderwidth=1
            )
        )
        return fig
    return None

In [7]:
%%writefile df_filter_utils.py

# Este archivo contiene funciones para filtrar y agrupar datos de manera interactiva en Streamlit.

# Función: search_and_filter_interface
# - Proporciona una interfaz para buscar y filtrar el DataFrame por columna, rango numérico, fechas o valores de texto.
# - Usa AgGrid para mostrar los resultados filtrados de manera interactiva.
# - Permite aplicar filtros complejos y ver los resultados en tiempo real.

# Función: groupby_interface
# - Permite al usuario agrupar y resumir los datos por una o más columnas y aplicar funciones de agregación (suma, media, etc.).
# - Muestra el resultado en una tabla interactiva.
# - Es útil para obtener resúmenes personalizados de los datos filtrados.

def search_and_filter_interface(df_search, columnas_contables=[], columnas_forzar_fecha=[], columnas_forzar_str=[], columnas_forzar_num=[]
                            , include_numeric=True):

    import streamlit as st
    import pandas as pd
    from st_aggrid import AgGrid, GridOptionsBuilder, JsCode
    import math
    from turtle import width
    import plotly.graph_objects as go
    import colorsys
    import numpy as np  
    from df_filter_utils import groupby_interface

    # --- formateador en JS -------
    currency_fmt = JsCode("""
    function(params) {
        if (params.value === null || params.value === undefined) {
            return '';
        }
        return '$ ' + Number(params.value)
            .toLocaleString(undefined, {minimumFractionDigits: 2, maximumFractionDigits: 2});
    }
    """)

    df = df_search.copy()

    col1, col2, col3, space = st.columns([2, 5, 2, 3])
    with col1:
        st.markdown("Buscar en Columna:")
        if include_numeric:
            # Incluir columnas numéricas en el selectbox
            columnas_disponibles = df.columns.tolist()
        else:
            # Excluir columnas numéricas
            columnas_disponibles = [col for col in df.columns if not pd.api.types.is_numeric_dtype(df[col])]

        column = st.selectbox("", columnas_disponibles, key="col_select", label_visibility="collapsed")
        
    with col2:
        # Determinar tipo de columna (forzado o automático)
        if column in columnas_forzar_fecha:
            tipo = "fecha"
        elif column in columnas_forzar_num:
            tipo = "num"
        elif column in columnas_forzar_str:
            tipo = "str"
        else:
            if pd.api.types.is_numeric_dtype(df[column]):
                tipo = "num"
            elif pd.api.types.is_datetime64_any_dtype(df[column]):
                tipo = "fecha"
            elif pd.api.types.is_timedelta64_dtype(df[column]):
                tipo = "num"
            else:
                tipo = "str"

        if tipo == "num":
            st.markdown("Selecciona rango numérico:")
            min_val = math.floor(df[column].min())
            max_val = math.ceil(df[column].max())
            # Evitar error de rango inválido
            if min_val == max_val:
                valor = (min_val, max_val)
                st.info(f"Solo existe un valor posible: {min_val}")
            else:
                valor = st.slider(
                    "",
                    min_value=min_val,
                    max_value=max_val,
                    value=(min_val, max_val),
                    step=1,
                    key="num_slider",
                    label_visibility="collapsed"
                )
            # Si es la columna de duración en horas, mostrar conversión
            if column.lower().startswith("duración viaje") and "hrs" in column.lower():
                st.caption(
                    f"Rango seleccionado: "
                    f"{horas_a_dhm(valor[0])} → {horas_a_dhm(valor[1])}"
                )
        elif tipo == "fecha":
            min_date = pd.to_datetime(df[column]).min().date()
            max_date = pd.to_datetime(df[column]).max().date()
            st.markdown("Selecciona rango de fechas:")
            col_fecha1, col_fecha2 = st.columns(2)
            with col_fecha1:
                fecha1 = st.date_input("Fecha inicio", value=min_date, min_value=min_date, max_value=max_date, key="fecha_inicio", label_visibility="collapsed")
            with col_fecha2:
                fecha2 = st.date_input("Fecha fin", value=max_date, min_value=min_date, max_value=max_date, key="fecha_fin", label_visibility="collapsed")
            # Lógica para determinar los valores efectivos
            if fecha1 is None and fecha2 is None:
                fecha1 = min_date
                fecha2 = max_date
            elif fecha1 is None:
                fecha1 = min_date
            elif fecha2 is None:
                fecha2 = max_date
            valor = (fecha1, fecha2)
        else:
            st.markdown("Valor a buscar:")
            unique_options = sorted(df[column].dropna().astype(str).unique().tolist())
            valor = st.multiselect(
                "",
                options=unique_options,
                default=[],
                key="text_input",
                label_visibility="collapsed"
            )
            if not valor:
                st.caption("Opciones: " + ", ".join(unique_options[:20]) + (" ..." if len(unique_options) > 20 else ""))
            else:
                st.caption("Filtrando por: " + ", ".join(valor))

    with col3:
        st.markdown("Confirmar:")
        aplicar = st.button("Aplicar Filtro", key="aplicar_btn")

    # Solo filtra al presionar el botón
    if 'filtro' not in st.session_state or aplicar:
        if tipo == "num":
            filtro = df[(df[column] >= valor[0]) & (df[column] <= valor[1])]
        elif tipo == "fecha":
            filtro = df[
                (pd.to_datetime(df[column]).dt.date >= valor[0])
                & (pd.to_datetime(df[column]).dt.date <= valor[1])
            ]
        else:
            if valor:
                filtro = df[df[column].astype(str).isin(valor)]
            else:
                filtro = df
        st.session_state.filtro = filtro
    else:
        filtro = st.session_state.filtro
        
    #groupby_interface(filtro)

    # Configuración visual contable
    gb = GridOptionsBuilder.from_dataframe(filtro)
    for col in filtro.columns:
        gb.configure_column(col, filter=False, resizable=True, sortable=True, wrapText=True)
        if col in columnas_contables:
            gb.configure_column(
                col,
                type=["numericColumn", "rightAligned"],
                valueFormatter=currency_fmt,
                cellStyle={'text-align': 'right'}
            )
    #gb.configure_pagination(paginationAutoPageSize=False, paginationPageSize=30)
    gridOptions = gb.build()

    df_final = AgGrid(
        filtro,
        gridOptions=gridOptions,
        enable_enterprise_modules=True,
        allow_unsafe_jscode=True,
        update_mode='MODEL_CHANGED',
        height=380
    )

    df_filtrado_en_aggrid = pd.DataFrame(df_final['data'])

    return df_filtrado_en_aggrid
        
def groupby_interface(df):
    
    import streamlit as st
    import pandas as pd
    import numpy as np
    from st_aggrid import AgGrid, GridOptionsBuilder, JsCode
    from turtle import width
    import math

    with st.expander("Agrupar y resumir información (opcional)", expanded=False):

        opciones_agrupado = [col for col in ["Periodo", "Tracto"] if col in df.columns]
        opciones_resumen = [col for col in ["kmstotales"] if col in df.columns]

        agrupado = st.multiselect(
            "¿Por qué quieres agrupar la información?",
            options=df.columns.tolist(),
            default=opciones_agrupado,
            key="agrupado"
        )

        opciones_para_resumir = [col for col in df.columns if col not in agrupado]
        resumen = st.multiselect(
            "¿Qué datos quieres resumir? (por ejemplo, kilómetros, costos, etc)",
            options=opciones_para_resumir,
            default=opciones_resumen,
            key="resumen"
        )

        funciones = {}
        opciones_funcion = ["Suma", "Promedio", "Mínimo", "Máximo", "Cantidad", "Valores únicos", "Mediana", "Desviación estándar"]
        equivalencias = {
            "Suma": "sum",
            "Promedio": "mean",
            "Mínimo": "min",
            "Máximo": "max",
            "Cantidad": "count",
            "Valores únicos": "nunique",
            "Mediana": "median",
            "Desviación estándar": "std"
        }
        for dato in resumen:
            funcion = st.selectbox(
                f"¿Cómo quieres resumir {dato}?",
                options=opciones_funcion,
                index=0 if dato == "kmstotales" else 1,
                key=f"funcion_{dato}"
            )
            funciones[dato] = equivalencias[funcion]

        if agrupado and funciones and resumen:
            # Solo aplica funciones numéricas a columnas numéricas
            funciones_validas = {}
            for col, func in funciones.items():
                if func in ["sum", "mean", "min", "max", "median", "std"]:
                    if pd.api.types.is_numeric_dtype(df[col]):
                        funciones_validas[col] = func
                else:
                    funciones_validas[col] = func
            if funciones_validas:
                resultado = df.groupby(agrupado).agg(funciones_validas).reset_index()
                st.dataframe(resultado)
            else:
                st.warning("No hay columnas numéricas seleccionadas para las funciones de agregación numérica.")
        else:
            st.info("Selecciona al menos una opción para agrupar y una para resumir para ver el resultado.")



In [8]:
%%writefile comparar_comp_utils.py

# Este archivo contiene funciones para comparar los componentes de CPK entre diferentes periodos y criterios.

# Función: construir_df_cpk_periodo
# - Construye un DataFrame con los valores de CPK y otros indicadores por periodo y grupo de análisis.
# - Facilita la comparación entre diferentes métodos de cálculo.

# Función: construir_titulo
# - Genera un título legible para los gráficos de comparación, basado en la variable y los periodos seleccionados.

# Función: labels_arriba
# - Calcula la posición vertical para los labels de los gráficos, evitando que se sobrepongan.

# Función: comparar_componentes_cpk
# - Genera un gráfico de barras con error para comparar la media y desviación estándar de los componentes de CPK.
# - El formato de los labels respeta el parámetro es_dinero para mostrar o no el signo $.
# - Es clave para el análisis visual comparativo de los indicadores de rendimiento.

def construir_df_cpk_periodo(df_all, grupos=['todas', 'costo', 'componente', 'cargas']):
    
    """
    Construye un DataFrame de CPK por periodo según los grupos seleccionados.
    Args:
        df_all: DataFrame fuente con columnas de CPK por grupo.
        grupos: lista de grupos a incluir. Opciones:
            'todas'      -> Todas las Órdenes
            'costo'      -> Órdenes con costo
            'componente' -> Órdenes con el Componente
            'cargas'     -> Por Cargas
    Returns:
        df_cpk_periodo: DataFrame con los CPK seleccionados.
    """
    
    import pandas as pd

    columnas = {}
    if 'todas' in grupos:
        columnas.update({
            'CPK Combustible (Todas las Órdenes)': df_all['CPK Combustible (Todas las Órdenes)'],
            'CPK Peajes (Todas las Órdenes)': df_all['CPK Peajes (Todas las Órdenes)'],
            'Total CPK (Todas las Órdenes)': (
                df_all['CPK Combustible (Todas las Órdenes)'] +
                df_all['CPK Peajes (Todas las Órdenes)']
            ),
            'Rendimiento Kms/Litro (Todas las Órdenes)': df_all['Rendimiento Kms/Litro (Todas las Órdenes)'],
            'Costo por Litro (Todas las Órdenes)': df_all['Costo por Litro (Todas las Órdenes)'],
            'No. Órdenes Consideradas (Todas las Órdenes)': df_all['No. Órdenes Consideradas (Todas las Órdenes)'],
        })
    if 'costo' in grupos:
        columnas.update({
            'CPK Combustible (Órdenes con costo)': df_all['CPK Combustible (Órdenes con costo)'],
            'CPK Peajes (Órdenes con costo)': df_all['CPK Peajes (Órdenes con costo)'],
            'Total CPK (Órdenes con costo)': (
                df_all['CPK Combustible (Órdenes con costo)'] +
                df_all['CPK Peajes (Órdenes con costo)']
            ),
            'Rendimiento Kms/Litro (Órdenes con costo)': df_all['Rendimiento Kms/Litro (Órdenes con costo)'],
            'Costo por Litro (Órdenes con costo)': df_all['Costo por Litro (Órdenes con costo)'],
            'No. Órdenes Consideradas (Órdenes con costo)': df_all['No. Órdenes Consideradas (Órdenes con costo)'],
        })
    if 'componente' in grupos:
        columnas.update({
            'CPK Combustible (Órdenes con el Componente)': df_all['CPK Combustible (Órdenes con Componente)'],
            'CPK Peajes (Órdenes con el Componente)': df_all['CPK Peajes (Órdenes con Componente)'],
            'Total CPK (Órdenes con el Componente)': (
                df_all['CPK Combustible (Órdenes con Componente)'] +
                df_all['CPK Peajes (Órdenes con Componente)']
            ),
            'Rendimiento Kms/Litro (Órdenes con el Componente)': df_all['Rendimiento Kms/Litro (Órdenes con Componente)'],
            'Costo por Litro (Órdenes con el Componente)': df_all['Costo por Litro (Órdenes con Componente)'],
            'No. Órdenes Consideradas (Órdenes con Combustible)': df_all['No. Órdenes Consideradas (Órdenes con Combustible)'],
        })
    if 'cargas' in grupos:
        columnas.update({
            'CPK Combustible (Entre Cargas)': df_all['CPK Combustible (Entre Cargas)'],
            'CPK Peajes (Entre Cargas)': df_all['CPK Peajes (Entre Cargas)'],
            'Total CPK (Entre Cargas)': (
                df_all['CPK Combustible (Entre Cargas)'] +
                df_all['CPK Peajes (Entre Cargas)']
            ),
            'Rendimiento Kms/Litro (Entre Cargas)': df_all['Rendimiento Kms/Litro (Entre Cargas)'],
            'Costo por Litro (Entre Cargas)': df_all['Costo por Litro (Entre Cargas)'],
            'No. Cargas con Combustible': df_all['No. Cargas con Combustible'],
            'No. Cargas con Peajes': df_all['No. Cargas con Peajes'],
            'No. Cargas con Mantenimiento': df_all['No. Cargas con Mantenimiento'],
        })
    df_cpk_periodo = pd.DataFrame(columnas)
    df_cpk_periodo.index.name = 'Periodo'
    return df_cpk_periodo

# --- Construcción automática del título ---
def construir_titulo(variable, periodos):
    # Variable legible
    var_legible = variable.replace('CPK', 'CPK').replace('Total', 'Total').replace('(', '').replace(')', '')
    # Periodos legibles
    if len(periodos) == 1:
        periodo_str = f"{periodos[0]}"
    else:
        periodo_str = f"{periodos[0]} a {periodos[-1]}"
    return f"{var_legible}"

# --- Cálculo de posiciones para los labels (por encima del error) ---
def labels_arriba(media, std, decimales=3, sep=1):
    return [m + s + sep*max(media+std) for m, s in zip(media, std)]

def comparar_componentes_cpk(df_cpk_periodo, variable='CPK Combustible', periodos=None, width=800, height=900, es_dinero=False):
    import plotly.graph_objects as go
    import pandas as pd
    import numpy as np

    if periodos is None:
        periodos = df_cpk_periodo.index.tolist()


    cols = df_cpk_periodo.filter(like=variable).columns

    # Limpia NaN e inf antes de calcular media y std
    datos = df_cpk_periodo.loc[periodos, cols].replace([np.inf, -np.inf], np.nan).dropna()

    if datos.empty:
        import streamlit as st
        st.warning(f"No hay datos válidos para la variable '{variable}' en los periodos seleccionados.")
        return go.Figure()  # Devuelve una figura vacía para evitar el error

    media = datos.mean().round(2)
    std = datos.std().round(2).fillna(0)  # <-- Esto pone 0 donde std es NaN

    # Decide el formato del label según es_dinero
    if es_dinero:
        labels = [
            f"${m:.2f} ± ${s:.2f}" if not np.isnan(s) else f"${m:.2f} ± N/A"
            for m, s in zip(media.values, std.values)
        ]
    else:
        labels = [
            f"{m:.2f} ± {s:.2f}" if not np.isnan(s) else f"{m:.2f} ± N/A"
            for m, s in zip(media.values, std.values)
        ]

    PALETA = ['#4361EE', '#5086F2', '#F9C74F', '#222']

    # Mapeo de nombres de columna a nombres cortos de grupo
    grupo_map = {
        "Todas las Órdenes": "Todas las Órdenes",
        "Órdenes con costo": "Órdenes con costo",
        "Órdenes con el Componente": "Órdenes con el Componente",
        "Entre Cargas": "Entre Cargas"
    }
    # Busca el grupo en el nombre de la columna
    def extraer_grupo(col):
        for k in grupo_map:
            if k in col:
                return grupo_map[k]
        return col  # fallback

    labels_x = [extraer_grupo(col) for col in media.index]

    y_label_pos = labels_arriba(media.values, std.values, decimales=3, sep=0.08)

    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=labels_x,
        y=media.values,
        error_y=dict(
            type='data',
            array=std.values,
            color='black',
            thickness=2.5,
            width=10
        ),
        marker_color=PALETA[:len(media)],
        opacity=0.92,
        name='Media ± Desv. Estándar'
    ))

    # Calcula el rango de y para definir un umbral de separación mínima
    y_range = max(y_label_pos) - min(y_label_pos)
    min_sep = y_range * 0.07  # 7% del rango, ajusta según lo que veas visualmente

    # Guarda las posiciones ya usadas para comparar
    used_y = []

    for xi, yi, label in zip(labels_x, y_label_pos, labels):
        # Busca cuántos labels ya están cerca de este yi
        shift_count = 0
        for y_prev in used_y:
            if abs(yi - y_prev) < min_sep:
                shift_count += 1
        used_y.append(yi)
        fig.add_annotation(
            x=xi,
            y=yi + shift_count * min_sep,  # Sube el label si está muy cerca de otro
            text=label,
            showarrow=False,
            font=dict(size=13, color="#222", family="Arial"),
            yanchor="bottom"
        )

    fig.update_layout(
        title=construir_titulo(variable, periodos),
        xaxis_title='Grupo',
        yaxis_title='Valor',
        template='plotly_white',
        width=width,
        height=height,
        margin=dict(t=200, l=100, b=70, r=40),
        font=dict(size=16, family='Arial'),
        xaxis=dict(
        tickangle=-30)  
    )

    return fig

In [9]:
%%writefile tracto_utils.py

# Este archivo contiene funciones para analizar y visualizar información específica de cada tracto.

# Función: monocromatic_color
# - Genera colores monocromáticos derivados de un color base para distinguir visualmente diferentes variables.

# Función: plot_acumulado_vs_kms
# - Grafica la evolución acumulada de costos y kilómetros para uno o varios tractos.
# - Permite comparar el desempeño de los tractos a lo largo del tiempo.

# Función: plot_costos_vs_kms_bars
# - Muestra barras comparativas de los costos y kilómetros totales para un tracto en un periodo dado.

# Función: seccion_graficos_tracto
# - Orquesta la visualización de los gráficos y tablas para un tracto seleccionado en la app Streamlit.
# - Incluye gráficos de acumulados, barras y el historial de cargas.
# - Facilita el análisis detallado y visual de cada tracto.

from turtle import width
import pandas as pd
import plotly.graph_objects as go
import colorsys

def monocromatic_color(base_hex, idx, total):
    base_rgb = tuple(int(base_hex.lstrip('#')[i:i+2], 16)/255. for i in (0, 2, 4))
    h, s, v = colorsys.rgb_to_hsv(*base_rgb)
    if total == 1:
        v2 = v
    else:
        v2 = 0.6 + (0.4 * idx / max(total-1, 1))
        v2 = max(0.2, min(v2, 1.0))
    rgb = colorsys.hsv_to_rgb(h, s, v2)
    return '#{:02x}{:02x}{:02x}'.format(int(rgb[0]*255), int(rgb[1]*255), int(rgb[2]*255))

def plot_acumulado_vs_kms(df, tractos, title=None, width=800, height=600):

    TRACTO_BASE_COLORS = [
    "#1f77b4", "#d62728", "#2ca02c", "#ff7f0e", "#9467bd"
    ]
    LINE_STYLES = {
        "kmstotales": "solid",
        "Costo Combustible": "dot",
        "Costo Peajes": "dot",
        "Costo Mantenimiento": "dot",
    }
    MARKERS = {
        "kmstotales": "circle",
        "Costo Combustible": "diamond",
        "Costo Peajes": "square",
        "Costo Mantenimiento": "triangle-up",
    }
    VARIABLES = [
        ("kmstotales", "acum_kms", "kmstotales"),
        ("Costo Combustible", "acum_combustible", "Costo Combustible"),
        ("Costo Peajes", "acum_peajes", "Costo Peajes"),
        ("Costo Mantenimiento", "acum_mantenimiento", "Costo Mantenimiento"),
    ]
    YAXIS_MAP = {
        "Costo Combustible": "y1",
        "Costo Peajes": "y1",
        "Costo Mantenimiento": "y1",
        "kmstotales": "y2"
    }
    MARKER_SIZE = 6

    COMPONENTE_COLORES = {
        "kmstotales": "#F2CD5E",           # Amarillo
        "Costo Combustible": "#233ED9",    # Azul fuerte
        "Costo Peajes": "#B3C8F2",         # Azul medio
        "Costo Mantenimiento": "#5086F2",  # Azul claro
    }

    df = df[df['Tracto'].isin(tractos)].copy()
    df['Inicio de la Orden'] = pd.to_datetime(df['Inicio de la Orden'])
    df = df.sort_values('Inicio de la Orden')

    fig = go.Figure()
    for tracto_idx, tracto in enumerate(tractos):
        base_color = TRACTO_BASE_COLORS[tracto_idx % len(TRACTO_BASE_COLORS)]
        dft = df[df['Tracto'] == tracto].sort_values('Inicio de la Orden').copy()
        dft['acum_combustible'] = dft['Costo Combustible'].cumsum()
        dft['acum_peajes'] = dft['Costo Peajes'].cumsum()
        dft['acum_mantenimiento'] = dft['Costo Mantenimiento'].cumsum()
        dft['acum_kms'] = dft["kmstotales"].cumsum()
        for var_idx, (variable, var_acum, var_single) in enumerate(VARIABLES):
            # Si solo hay un tracto, usa la paleta de componentes
            if len(tractos) == 1:
                color = COMPONENTE_COLORES[variable]
            else:
                color = monocromatic_color(base_color, var_idx, len(VARIABLES))
            vals_acum = dft[var_acum]
            vals_puntual = dft[var_single]
            ordenes = dft.index
            customdata = pd.concat(
                [
                    vals_puntual,
                    pd.Series(ordenes, index=vals_puntual.index, name="No. Orden")
                ],
                axis=1
            ).values
            fig.add_trace(go.Scatter(
                x=dft['Inicio de la Orden'],
                y=vals_acum,
                name=f"Acum. {variable} ({vals_acum.iloc[-1]:,.0f}) | {tracto}",
                mode='lines+markers',
                line=dict(
                    color=color,
                    width=2,
                    dash=LINE_STYLES[variable]
                ),
                marker=dict(
                    size=MARKER_SIZE,
                    color=color,
                    symbol=MARKERS[variable],
                    line=dict(color="black", width=0.6)
                ),
                yaxis=YAXIS_MAP[variable],
                showlegend=True,
                legendgroup=None,
                customdata=customdata,
                hovertemplate=(
                    "<b>%{fullData.name}</b><br>"
                    "Fecha: %{x|%d-%b-%Y}<br>"
                    "Acumulado: <b>%{y:,}</b><br>"
                    "Valor puntual: %{customdata[0]:,.2f}<br>"
                    "No. Orden: %{customdata[1]}<extra></extra>"
                )
            ))

    fig.update_layout(
        title=title if title is not None else None,
        xaxis=dict(title='Fecha de Inicio de la Orden'),
        yaxis=dict(title='Acumulado de Costos ($)', rangemode='tozero'),
        yaxis2=dict(
            title='Acumulado KMs Totales',
            overlaying='y',
            side='right',
            rangemode='tozero',
            showgrid=False
        ),
        legend=dict(
            orientation="v",
            yanchor="top",
            y=1.0,
            xanchor="left",
            x=0.01,
            bgcolor="rgba(255,255,255,0.6)",
            bordercolor="lightgray",
            borderwidth=1,
            font=dict(size=13),
        ),
        height=height,
        width=width,
        margin=dict(t=150, l=60, b=60, r=60),
        plot_bgcolor="white"
    )
    return fig

def plot_costos_vs_kms_bars(df, fecha_inicio, fecha_fin, tracto, width=800, height=600):
    # Filtrado y suma
    data = df[(df['Inicio de la Orden'] >= fecha_inicio) & (df['Cierre de la Orden'] < fecha_fin) & (df['Tracto'] == tracto)].copy()
    total = data[['Costo Combustible', 'Costo Peajes', 'Costo Mantenimiento', 'kmstotales']].sum()

    colores = {
        'Costo Combustible': "#233ED9",    # Azul fuerte
        'Costo Peajes': "#B3C8F2",         # Azul medio
        'Costo Mantenimiento': "#5086F2",  # Azul claro
        'kmstotales': "#F2CD5E",           # Amarillo
    }

    fig = go.Figure()
    # Barras de costos
    for var in ['Costo Combustible', 'Costo Peajes', 'Costo Mantenimiento']:
        fig.add_trace(go.Bar(
            x=[var],
            y=[total[var]],
            name=var,
            marker_color=colores[var],
            yaxis='y1',
            hovertemplate=f"Tipo: {var}<br>Monto: $%{{y:,.2f}}<extra></extra>",
            text=[f"${total[var]:,.0f}"],  # Texto arriba de la barra
            textposition='outside'
        ))
    # Barra de kms (eje derecho)
    fig.add_trace(go.Bar(
        x=['kmstotales'],
        y=[total['kmstotales']],
        name='Kms Totales',
        marker_color=colores['kmstotales'],
        yaxis='y2',
        hovertemplate="Tipo: kmstotales<br>Kms: %{y:,.0f}<extra></extra>",
        text=[f"{total['kmstotales']:,.0f}"],
        textposition='outside'
    ))



    fig.update_layout(
        barmode='group',
        title=f"Acumulados de Costos y Kms | Tracto {tracto} | {fecha_inicio.strftime('%d-%b-%Y')} al {fecha_fin.strftime('%d-%b-%Y')}",
        xaxis_title='Variable',
        yaxis=dict(
            title='Costo ($)',
            rangemode='tozero'
        ),
        yaxis2=dict(
            title='Kms Totales',
            overlaying='y',
            side='right',
            rangemode='tozero',
            showgrid=False
        ),
        bargap=0.3,
        legend=dict(
            orientation="h",
            y=1.13,
            x=0.01
        ),
        template='plotly_white',
        height=height,
        width=width,
        margin=dict(t=150, l=60, b=60, r=60),
    )
    return fig

def seccion_graficos_tracto(df, historial_cargas, key=""):
    import streamlit as st
    from tracto_utils import plot_acumulado_vs_kms, plot_costos_vs_kms_bars
    import numpy as np
    from graph_hist_utils import streamlit_viz_selector,get_viz_figure
    from st_aggrid import AgGrid, GridOptionsBuilder

    st.markdown("""
    **Indicadores gráficos por tracto**

    Selecciona un tracto para visualizar la evolución acumulada de costos y kilómetros, así como el resumen total de cada componente para el periodo disponible.
    """)

    tractos = df['Tracto'].unique()
    tracto_default = tractos[0] if len(tractos) > 0 else None
    tracto_sel = st.selectbox("Selecciona un tracto", options=tractos, index=0, key=f"tracto_selector_{key}")

    fecha_inicio = df[df['Tracto'] == tracto_sel]['Inicio de la Orden'].min()
    fecha_fin = df[df['Tracto'] == tracto_sel]['Cierre de la Orden'].max()

    hist_cargas = historial_cargas[(historial_cargas['Tracto'] == tracto_sel) & (historial_cargas['Fecha Orden de Carga'] >= fecha_inicio) & (historial_cargas['Fecha Orden de Carga'] <= fecha_fin)]

    
    title = f"Acumulados de Costos y Kms | Tracto {tracto_sel} | {fecha_inicio.strftime('%d-%b-%Y')} al {fecha_fin.strftime('%d-%b-%Y')}"
    fig1 = plot_acumulado_vs_kms(df[(df['Inicio de la Orden']>=fecha_inicio) & (df['Cierre de la Orden']<fecha_fin)], [tracto_sel], title=title, width=400, height=700)
    st.plotly_chart(fig1, use_container_width=True)

    fig2 = plot_costos_vs_kms_bars(
        df,
        fecha_inicio=fecha_inicio,
        fecha_fin=fecha_fin,
        tracto=tracto_sel,
        width=400, 
        height=500
    )
    st.plotly_chart(fig2, use_container_width=True)

    st.markdown("### Historial de Cargas")
    st.info(
        """
        **¿Qué muestra esta tabla?**

        Aquí puedes ver el historial de cargas del tracto seleccionado.  
        Cada fila representa el periodo entre una carga de combustible y la siguiente.  

        Esta información te permite analizar en detalle el desempeño operativo y los costos de cada tracto entre cada abastecimiento.
        """
    )

    st.dataframe(
                hist_cargas.sort_values('Fecha Orden de Carga', ascending=True).reset_index(drop=True),
                use_container_width=True,
                height=420  # Cambia el valor según lo que necesites
            )

    st.markdown("### Visualización de Datos")
    st.info(
        "Selecciona una columna numérica y el tipo de gráfico para visualizar la distribución de los datos del historial de cargas. "
        "Puedes elegir entre barras (mediana, mínimo y máximo), boxplot o pastel por rangos. "
        "Esto te ayudará a analizar rápidamente la variabilidad y los valores típicos de la columna seleccionada."
    )
    
    seleccionada1, tipo_grafico1 = streamlit_viz_selector(hist_cargas, key=f"viz_tracto1{key}")
    if seleccionada1 and tipo_grafico1:
        fig1 = get_viz_figure(hist_cargas, seleccionada1, tipo_grafico1, width=400, height=500)
        if fig1:
            st.markdown(
                f"""
                <h4 style='text-align: center;'>
                    Gráfico de {tipo_grafico1} y {seleccionada1}
                </h4>
                """,
                unsafe_allow_html=True
            )
            st.plotly_chart(fig1, use_container_width=True)
        else:
            st.warning("No se pudo generar el gráfico con los datos seleccionados.")
        

In [ ]:
%%writefile app.py

# Este archivo es el punto de entrada principal de la aplicación Streamlit.
# Orquesta la carga de datos, la configuración de la app y la integración de todas las funciones y módulos anteriores.

# - Configura el layout y el título de la app.
# - Carga los datos y los prepara para el análisis.
# - Permite buscar, filtrar y explorar los datos de manera interactiva.
# - Muestra indicadores generales, gráficos de CPK, completitud, histogramas y comparativos entre tractos.
# - Integra todas las funciones utilitarias y de visualización para ofrecer una experiencia de análisis completa y flexible.

import streamlit as st
import pandas as pd
from st_aggrid import AgGrid, GridOptionsBuilder, JsCode
import math
from utils import show_info_columns, df_completitud, plot_completitud_y_mediana
from turtle import width
import plotly.graph_objects as go
import colorsys
from tracto_utils import seccion_graficos_tracto, plot_acumulado_vs_kms, plot_costos_vs_kms_bars, monocromatic_color
from historial_cargas import historial_entre_cargas
import numpy as np  
from calculos_cpk import agrupar_componentes_cpk, plot_cpk_barras_comparativo, cpk_desglosado
from df_filter_utils import search_and_filter_interface, groupby_interface
from graph_hist_utils import streamlit_viz_selector, get_viz_figure
from comparar_comp_utils import construir_df_cpk_periodo, comparar_componentes_cpk



def horas_a_dhm(horas):
    """Convierte horas (float) a una cadena con días, horas y minutos"""
    total_min = int(round(horas * 60))
    dias = total_min // (60 * 24)
    horas_rest = (total_min // 60) % 24
    minutos = total_min % 60
    partes = []
    if dias > 0:
        partes.append(f"{dias} día{'s' if dias != 1 else ''}")
    if horas_rest > 0 or dias > 0:
        partes.append(f"{horas_rest} h")
    return ", ".join(partes)


if __name__ == "__main__":

    st.set_page_config(page_title="Proyecto - PyTrack Analytics", layout="wide")

    if 'df' not in st.session_state:
        df = pd.read_excel('Base_viz.xlsx', index_col=0)

        df['Duración Viaje'] = df['Cierre de la Orden'] - df['Inicio de la Orden']

        if 'Duración Viaje (hrs)' not in df.columns and pd.api.types.is_timedelta64_dtype(df['Duración Viaje']):
            df['Duración Viaje (hrs)'] = (df['Duración Viaje'].dt.total_seconds() / 3600).round(2)

        df = df[['EC', 'Proyecto', 'Cliente', 'Tracto', 'Inicio de la Orden', 'Cierre de la Orden', 'Duración Viaje (hrs)', 'Edo. Origen', 'Edo. Destino', 'Cdad. Origen', 'Cdad. Destino', 'Ruta Estados', 'Ruta Ciudades',
                'Conductor', 'kmstotales', 'No. Remolques','Litros','Costo por litro', 'Costo Combustible', 'Costo Peajes', 'Costo Mantenimiento','Costo Total','CPK Orden', 'Periodo', 'Conteo',
                'lat_origen', 'lon_origen', 'lat_destino', 'lon_destino','Orden con Costo de Combustible','Orden con Costo de Peajes', 'Orden con Costo de Mantenimiento']].rename({'Conteo':'No. Viajes'}, axis = 1).copy()

        df.index.name = 'No. Orden'

        df.drop(['lat_origen', 'lon_origen', 'lat_destino', 'lon_destino'], axis=1, inplace=True, errors='ignore')

        df.reset_index(inplace=True)
        
        st.session_state.df = df.copy()

    if 'historial_cargas' and 'historial_cargas_grouped' not in st.session_state:
        historial_cargas, historial_cargas_grouped = historial_entre_cargas(st.session_state.df)
        st.session_state.historial_cargas = historial_cargas
        st.session_state.historial_cargas_grouped = historial_cargas_grouped


    # Ejemplo de columnas contables y forzadas
    columnas_contables = [
        "Costo Combustible", "Costo Peajes", "Costo Mantenimiento","Costo Total", "CPK Orden"
    ]
    columnas_forzar_fecha = ["Inicio de la Orden", "Cierre de la Orden"]
    columnas_forzar_str = ["Proyecto", "Cliente", "Tracto","No. Orden"]
    columnas_forzar_num = ["kmstotales", "No. Remolques", "Duración Viaje (hrs)"]

    # Título de la aplicación
    st.title("Bienvenido, TDR")
    st.markdown(""" """)

    st.markdown("""
    Esta aplicación te permite buscar y filtrar órdenes de transporte, visualizar datos y generar gráficos para análisis.
    Puedes buscar por columnas específicas, aplicar filtros y explorar los datos de manera interactiva.
    """)

    # Search bar y filtro
    st.subheader("Buscar y filtrar órdenes")

    with st.expander("Información de la sección", expanded=False):

        st.info(
            """
            **¿Cómo funciona la búsqueda y el filtrado?**

            - Selecciona una columna del listado para buscar información específica.
            - Dependiendo del tipo de columna, podrás:
            - Seleccionar un **rango de fechas** para filtrar por periodos.
            - Buscar uno o varios **valores de texto** (por ejemplo, tracto, cliente, proyecto).
            - Después de definir el filtro, haz clic en "Aplicar Filtro" para ver solo las órdenes que cumplen con los criterios seleccionados.
            - Puedes combinar varios filtros para afinar tu búsqueda y analizar subconjuntos de datos de interés.
            - Los resultados filtrados se mostrarán en la tabla inferior y se usarán en los indicadores y gráficos de la aplicación.

            Esta herramienta te permite explorar y analizar la información de manera flexible y personalizada.
            """
        )

    df_filtered = search_and_filter_interface(
        st.session_state.df,
        columnas_contables=columnas_contables,
        columnas_forzar_fecha=columnas_forzar_fecha,
        columnas_forzar_str=columnas_forzar_str,
        columnas_forzar_num=columnas_forzar_num,
        include_numeric=False
    )

    st.subheader("Resumen de las órdenes seleccionadas")

    st.info(
        "A continuación se muestran los indicadores generales y gráficos basados en los datos filtrados. "
        "Puedes explorar los datos de manera interactiva y obtener información valiosa sobre las órdenes de transporte.")

    # Mostrar indicadores generales
    show_info_columns(df_filtered)

    st.subheader("Análisis Desglosado de CPK por Componente")
    with st.expander("Información de la sección", expanded=False):
        st.info("""
            En la siguiente sección se muestra el **Costo por Kilómetro (CPK)** desglosado en dos componentes de costo operacional: **Peajes** y **Combustible**.  
            Realizamos estos cálculos de tres diferentes maneras para observar cómo el método de cálculo afecta el valor del CPK:

            1. **Considerando todas las órdenes:** Se incluyen todas las órdenes, tengan o no costos en cada componente.
            2. **Solo órdenes con algún costo:** Se consideran únicamente las órdenes que presentan algún costo total.
            3. **Solo órdenes con el componente:** Para cada componente, solo se consideran las órdenes que presentan ese componente (por ejemplo, solo las órdenes con costo de peajes para el CPK de peajes).
            4. **Entre Cargas:** Se calcula el CPK considerando los costos y kms recorridos entre carga y carga.

            **Nota:** Los cálculos de CPK "entre carga y carga" **no aplican los filtros seleccionados**, ya que estos podrían excluir órdenes necesarias para este tipo de análisis. Por ello, **siempre se consideran todas las órdenes disponibles** para calcular los indicadores entre cargas, asegurando así la consistencia y precisión de los resultados.

            Esto permite comparar cómo varía el CPK según el criterio de inclusión de órdenes y analizar la importancia de cada componente en el costo total.

            A continuación puedes ver la gráfica comparativa de CPK por periodo y por cada criterio.
            """)

    cpk_desglosado(df_filtered, historial_cargas=st.session_state.historial_cargas)

    with st.expander("Completitud de las órdenes seleccionadas", expanded=False):

        st.subheader("Completitud de las órdenes seleccionadas")

        st.info(
            "A continuación se muestra el porcentaje de órdenes que tienen costos de combustible, peajes y mantenimiento a lo largo del tiempo. "
            "Esto te ayudará a identificar la completitud de los datos y detectar posibles áreas de mejora en la recolección de información.")

        completitud_groupby = df_completitud(df_filtered)
        
        fig_completitud = plot_completitud_y_mediana(
                completitud_groupby,
                columnas_estadistica=[]
            )

        st.plotly_chart(fig_completitud, use_container_width=True)

    with st.expander("Exploración visual de las órdenes seleccionadas", expanded=False):

        st.subheader("Exploración visual de las órdenes seleccionadas")
        st.info(
            "Selecciona una columna numérica y el tipo de gráfico para visualizar la distribución de los datos filtrados. "
            "Puedes elegir entre barras (mediana, mínimo y máximo), boxplot o pastel por rangos. "
            "Esto te ayudará a analizar rápidamente la variabilidad y los valores típicos de la columna seleccionada."
        )

        col1, col2, col3 = st.columns([1,1,1])

        with col1:
            seleccionada1, tipo_grafico1 = streamlit_viz_selector(df_filtered, idx = 5, key = '1g')
            st.markdown(f"#### Gráfico de {tipo_grafico1} para **{seleccionada1}**")
            fig1 = get_viz_figure(df_filtered, seleccionada1, tipo_grafico1, width=700, height=700)

            if fig1 is not None:
                st.plotly_chart(fig1, use_container_width=True)
        
        with col2:
            seleccionada2, tipo_grafico2 = streamlit_viz_selector(df_filtered, idx = 6, key = '2g')
            st.markdown(f"#### Gráfico de {tipo_grafico2} para **{seleccionada2}**")
            fig2 = get_viz_figure(df_filtered, seleccionada2, tipo_grafico2, width=700, height=700)

            if fig2 is not None:
                st.plotly_chart(fig2, use_container_width=True)

        with col3:
            seleccionada3, tipo_grafico3 = streamlit_viz_selector(df_filtered, idx = 7, key = '3g')
            st.markdown(f"#### Gráfico de {tipo_grafico3} para **{seleccionada3}**")
            fig3 = get_viz_figure(df_filtered, seleccionada3, tipo_grafico3, width=700, height=700)

            if fig3 is not None:
                st.plotly_chart(fig3, use_container_width=True)

    with st.expander("Comparativo entre Tractos", expanded=False):

        st.subheader("Comparativo entre Tractos")
        st.info(
            "En esta sección se comparan los tractos que están presentes en la flota. "
            "Puedes seleccionar los tractos que deseas comparar y ver cómo se desempeñan en términos de costos y rendimiento. "
        )

        col1, col2 = st.columns([1, 1])
        with col1:
            seccion_graficos_tracto(st.session_state.df, historial_cargas=st.session_state.historial_cargas,key="1tracto")
        with col2:
            seccion_graficos_tracto(st.session_state.df, historial_cargas=st.session_state.historial_cargas,key="2tracto")


In [14]:
# Esta celda ejecuta la aplicación Streamlit en modo headless en el puerto 8598.
# Permite lanzar la app desde el notebook o VSCode para pruebas y visualización.

! streamlit run app.py --server.port 8598 --server.headless true --browser.gatherUsageStats false

2025-06-14 16:58:34.812 Port 8598 is already in use
